# How to download necessary libraries (Only run once per device)

In [ ]:
pip install pyreadstat
pip install pandas
pip install matplotlib
pip intall pyforest

# Importing Libraries and Defining Functions

In [ ]:
#Final_loading_code
import os
import pandas as pd
import pyreadstat
import warnings
warnings.filterwarnings('ignore')

def load_files(file_paths, sheet_indices=None, sheet_name=None, return_meta=False):
    """
    Smart loader that handles multiple files and multiple sheets inside Excel files.
    
    Parameters:
    - file_paths (str or list): Single file path or list of file paths.
    - sheet_indices (list of int, optional): For Excel files, indices of sheets to load separately.
    - sheet_name (str/int, optional): Specific sheet to load from Excel (if only one).
    - return_meta (bool): Return (DataFrame, metadata) tuple for .sav files if True.

    Returns:
    - list: List of DataFrames or list of (DataFrame, metadata) tuples.
    """
    
    if isinstance(file_paths, str):
        file_paths = [file_paths]  # make it a list for consistency
    
    loaded_data = []
    
    for file_path in file_paths:
        if not os.path.exists(file_path):
            print(f"Error: File not found -> {file_path}")
            loaded_data.append((None, None) if return_meta else None)
            continue

        file_extension = os.path.splitext(file_path)[-1].lower()
        
        try:
            if file_extension == ".sav":
                df, meta = pyreadstat.read_sav(file_path)
                print(f"Loaded SPSS file: {file_path}")
                loaded_data.append((df, meta) if return_meta else df)
            
            elif file_extension == ".csv":
                df = pd.read_csv(file_path)
                print(f"Loaded CSV file: {file_path}")
                loaded_data.append((df, None) if return_meta else df)
            
            elif file_extension in [".xls", ".xlsx"]:
                excel_file = pd.ExcelFile(file_path)
                sheet_names = excel_file.sheet_names
                
                if sheet_indices is not None:
                    for idx in sheet_indices:
                        if idx < len(sheet_names):
                            df = excel_file.parse(sheet_name=sheet_names[idx])
                            print(f"Loaded sheet '{sheet_names[idx]}' from {file_path}")
                            loaded_data.append((df, None) if return_meta else df)
                        else:
                            print(f"Warning: Sheet index {idx} out of range in {file_path}")
                            loaded_data.append((None, None) if return_meta else None)

                elif sheet_name is not None:
                    df = pd.read_excel(file_path, sheet_name=sheet_name)
                    print(f"Loaded sheet '{sheet_name}' from {file_path}")
                    loaded_data.append((df, None) if return_meta else df)

                else:
                    df = pd.read_excel(file_path, sheet_name=None)
                    print(f"Loaded all sheets from {file_path}")
                    loaded_data.append((df, None) if return_meta else df)

            else:
                print(f"Unsupported file type: {file_extension}")
                loaded_data.append((None, None) if return_meta else None)
        
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
            loaded_data.append((None, None) if return_meta else None)
    
    return loaded_data


In [ ]:
file_list_1 = ["ALS_Young_2017-18.sav",
             "ALS_Young_2018-19.sav",
             "ALS_Young_2019-20.sav",
]

file_list_2=["ALS_Young_2020-21.sav",
             "ALS_Young_2021-22.sav",
             "ALS_Youn_202223.sav"]

In [ ]:
adf1, adf2, adf3= load_files(file_list_1)

In [ ]:
adf4, adf5, adf6= load_files(file_list_2)

In [ ]:
adf6

# Basic Pandas Methods

| **No.** | **Command** | **Description** |
|:-------:|:------------|:----------------|
| 1 | `dataset.head(10)` | Returns the first 10 entries of the dataset. |
| 2 | `dataset.tail(10)` | Returns the last 10 entries of the dataset. |
| 3 | `dataset.columns` | Lists all the column names in the dataset as a list. |
| 4 | `dataset['categorical_column'].value_counts()` | Returns the count of each unique category in the specified column. |
| 5 | `dataset['categorical_column'].nunique()` | Returns the number of unique categories in the specified column. |
| 6 | `dataset.to_csv("desired_csv_name.csv")` | Saves the dataset to a CSV file. |


In [ ]:
adf1.head(10)

In [ ]:
adf2.tail(10)

In [ ]:
adf1.columns

In [ ]:
adf1['CSP_name'].value_counts()

In [ ]:
adf1['CSP_name'].nunique()

# Function 2: find_columns()

## Implementing function 2

In [ ]:
def find_columns(df, pattern):
    return df.columns[df.columns.to_series().str.contains(pattern, case=False, regex=True)].tolist()


In [ ]:
gender_pattern=r"gend"      # For identifying gender related variables
sport_pattern = r"rugby"    # For identifying variables which belong to a particular sport
age_pattern = r"age"        # For identifying age related variables
location_pattern = r"county|region|state|district|area|location|csp|la_"  # For region/county related variables.
                                                                          #In ALS Adults data "csp" and "la" denote the region.

In [ ]:
als_df_1=[adf1,adf2,adf3]
als_df_2=[adf4, adf5, adf6]

als_years_1 = ['2017-18', '2018-19', '2019-20']
als_years_2 = ['2020-21', '2021-22', '2022-23'] 

In [ ]:
for df, year in zip(als_df_1, als_years_1):
    relevant_sport_columns = find_columns(df, sport_pattern)
    relevant_age_columns = find_columns(df, age_pattern)
    relevant_location_columns = find_columns(df, location_pattern)
    relevant_gender_columns = find_columns(df, gender_pattern)
    
    print(f"ALS_Data for {year}:")
    print("Sport Columns:", relevant_sport_columns)
    print("\n Age Columns:", relevant_age_columns)
    print("\n Location Columns:", relevant_location_columns)
    print("\n Gender Columns:", relevant_gender_columns)
    print("-" * 40)

In [ ]:
for df, year in zip(als_df_2, als_years_2):
    relevant_sport_columns = find_columns(df, sport_pattern)
    relevant_age_columns = find_columns(df, age_pattern)
    relevant_location_columns = find_columns(df, location_pattern)
    relevant_gender_columns = find_columns(df, gender_pattern)
    
    print(f"ALS_Data for {year}:")
    print("Sport Columns:", relevant_sport_columns)
    print("\n Age Columns:", relevant_age_columns)
    print("\n Location Columns:", relevant_location_columns)
    print("\n Gender Columns:", relevant_gender_columns)
    print("-" * 40)

### User Input

In [ ]:
# generic_columns=["Respondent_Serial","wt_gross"]
# gender_column='gend3'
# sport_column = "onceawk_modplus_everywhere_GR_RUGBY_CC018"
# age_column = "age_11"

# location_columns_1718 = ["Region_name","CSP_name"]
# location_columns_1819 = ["Region_name","CSP_name"]
# location_columns_1920 = ["Region_name","CSP_name2019"]

In [ ]:
generic_columns=["Respondent_Serial","wt_gross"]
gender_column='gend3'
sport_column = "onceawk_modplus_everywhere_GR_RUGBY_CC018"
age_column = "age_11"

location_columns_2021 = ["Region_name","CSP_name2020"]
location_columns_2122 = ["Region_name","CSP_name2020"]
location_columns_2223 = ["Region_name","CSP_name2019"]

# Function 3: filter_desired_column()

In [ ]:
def filter_desired_columns(data, *columns):
    selected_columns = []

    for col in columns:
        if isinstance(col, list):  # If col is a list, add all items to selected_columns
            selected_columns.extend(col)
        else:  # If col is a single column name, add it directly
            selected_columns.append(col)
    
    try:
        return data[selected_columns]  # Filter DataFrame with selected columns
    except KeyError as e:
        print(f"Error: Some columns not found in DataFrame - {e}")
        return None 

## Implementing Function 3

In [ ]:
# filtered_df_1718=filter_desired_columns(adf1, 
#                                         generic_columns,
#                                         sport_column, 
#                                         age_column, 
#                                         gender_column,
#                                         location_columns_1718)

# filtered_df_1819=filter_desired_columns(adf2,
#                                         generic_columns, 
#                                         sport_column, 
#                                         age_column, 
#                                         gender_column, 
#                                         location_columns_1819)

# filtered_df_1920=filter_desired_columns(adf3,
#                                         generic_columns, 
#                                         sport_column,
#                                         age_column, 
#                                         gender_column, 
#                                         location_columns_1920)

In [ ]:
filtered_df_2021=filter_desired_columns(adf4, 
                                        generic_columns,
                                        sport_column, 
                                        age_column, 
                                        gender_column,
                                        location_columns_2021)

filtered_df_2122=filter_desired_columns(adf5,
                                        generic_columns, 
                                        sport_column, 
                                        age_column, 
                                        gender_column, 
                                        location_columns_2122)

filtered_df_2223=filter_desired_columns(adf6,
                                        generic_columns, 
                                        sport_column,
                                        age_column, 
                                        gender_column, 
                                        location_columns_2223)

# Function 4: map_numerical_data_to_labels() 

In [ ]:
def map_numerical_data_to_labels(data,column_name="x" ,new_name="y",mapping={}):
    data[new_name]=data[column_name].map(mapping)
    data.drop(column_name, axis=1, inplace=True)
    return data

In [ ]:
csp_mapping = {
    1.0: "Bedfordshire & Luton",
    2.0: "Berkshire",
    3.0: "Birmingham",
    4.0: "Black Country",
    5.0: "Buckinghamshire and Milton Keynes",
    6.0: "Peterborough & Cambridgeshire",
    7.0: "Cheshire",
    8.0: "Cornwall and Isles of Scilly",
    9.0: "Cumbria",
    10.0: "Derbyshire",
    11.0: "Devon",
    12.0: "Dorset",
    13.0: "Durham",
    14.0: "Greater Essex",
    15.0: "Gloucestershire",
    16.0: "Greater Manchester",
    17.0: "Hampshire and Isle of Wight",
    18.0: "Herefordshire and Worcestershire",
    19.0: "Hertfordshire",
    20.0: "Humber",
    21.0: "Kent",
    22.0: "Lancashire",
    23.0: "Leicester, Leicestershire and Rutland",
    24.0: "Lincolnshire",
    25.0: "London",
    26.0: "Merseyside",
    27.0: "Norfolk",
    28.0: "North Yorkshire",
    29.0: "Northamptonshire",
    30.0: "Northumberland",
    31.0: "Nottinghamshire",
    32.0: "Oxfordshire",
    33.0: "Shropshire and Telford and the Wrekin",
    34.0: "Somerset",
    35.0: "South Yorkshire",
    36.0: "Staffordshire and Stoke-on-Trent",
    37.0: "Suffolk",
    38.0: "Surrey",
    39.0: "Sussex",
    40.0: "Tees Valley",
    41.0: "Tyne and Wear",
    42.0: "Coventry, Solihull & Warwickshire",
    43.0: "Bristol and West of England",
    44.0: "West Yorkshire",
    45.0: "Wiltshire & Swindon"
}


In [ ]:
region_mapping = {
    1.0: "East",
    2.0: "East Midlands",
    3.0: "London",
    4.0: "North East",
    5.0: "North West",
    6.0: "South East",
    7.0: "South West",
    8.0: "West Midlands",
    9.0: "Yorkshire and the Humber"
}

In [ ]:
csp_1920 = {
    -1.0: "No information",
    1.0: "Bedfordshire & Luton",
    2.0: "Berkshire",
    3.0: "Birmingham",
    4.0: "Black Country",
    5.0: "Buckinghamshire and Milton Keynes",
    6.0: "Peterborough & Cambridgeshire",
    7.0: "Cheshire",
    8.0: "Cornwall and Isles of Scilly",
    9.0: "Cumbria",
    10.0: "Derbyshire",
    11.0: "Devon",
    12.0: "Dorset",
    13.0: "Durham",
    14.0: "Greater Essex",
    15.0: "Gloucestershire",
    16.0: "Greater Manchester",
    17.0: "Hampshire and Isle of Wight",
    18.0: "Herefordshire and Worcestershire",
    19.0: "Hertfordshire",
    20.0: "Humber",
    21.0: "Kent",
    22.0: "Lancashire",
    23.0: "Leicester, Leicestershire and Rutland",
    24.0: "Lincolnshire",
    25.0: "London",
    26.0: "Merseyside",
    27.0: "Norfolk",
    28.0: "North Yorkshire",
    29.0: "Northamptonshire",
    30.0: "Northumberland",
    31.0: "Nottinghamshire",
    32.0: "Oxfordshire",
    33.0: "Shropshire and Telford and the Wrekin",
    34.0: "Somerset",
    35.0: "South Yorkshire",
    36.0: "Staffordshire and Stoke-on-Trent",
    37.0: "Suffolk",
    38.0: "Surrey",
    39.0: "Sussex",
    40.0: "Tees Valley",
    41.0: "Tyne and Wear",
    42.0: "Coventry, Solihull & Warwickshire",
    43.0: "Bristol and West of England",
    44.0: "West Yorkshire",
    45.0: "Wiltshire & Swindon",
    46.0: "Derbyshire and Nottinghamshire",
    47.0: "West and South Yorkshire",
    48.0: "Rise North East"
}

In [ ]:
map_numerical_data_to_labels(filtered_df_1718,column_name="CSP_name",new_name="CSP", mapping=csp_mapping)   
map_numerical_data_to_labels(filtered_df_1718,column_name="Region_name",new_name="Region", mapping=region_mapping)

map_numerical_data_to_labels(filtered_df_1819,column_name="CSP_name",new_name="CSP", mapping=csp_mapping)   
map_numerical_data_to_labels(filtered_df_1819,column_name="Region_name",new_name="Region", mapping=region_mapping)

map_numerical_data_to_labels(filtered_df_1920,column_name="CSP_name2019",new_name="CSP", mapping=csp_1920)   
map_numerical_data_to_labels(filtered_df_1920,column_name="Region_name",new_name="Region", mapping=region_mapping)

In [ ]:
map_numerical_data_to_labels(filtered_df_2021,column_name="CSP_name2020",new_name="CSP", mapping=csp_1920)   
map_numerical_data_to_labels(filtered_df_2021,column_name="Region_name",new_name="Region", mapping=region_mapping)

map_numerical_data_to_labels(filtered_df_2122,column_name="CSP_name2020",new_name="CSP", mapping=csp_1920)   
map_numerical_data_to_labels(filtered_df_2122,column_name="Region_name",new_name="Region", mapping=region_mapping)

map_numerical_data_to_labels(filtered_df_2223,column_name="CSP_name2019",new_name="CSP", mapping=csp_mapping)   
map_numerical_data_to_labels(filtered_df_2223,column_name="Region_name",new_name="Region", mapping=region_mapping)

# Optional Step (Can Skip)

In [ ]:
# 2017-18
filtered_df_1718 = filtered_df_1718.rename(columns={"age_11": "Age"})
filtered_df_1718["Age"] = filtered_df_1718["Age"].astype(str)
filtered_df_1718 = filtered_df_1718.join(
    filtered_df_1718["Age"].str.get_dummies(sep=",").add_prefix("Age_")
)

# 2018-19
filtered_df_1819 = filtered_df_1819.rename(columns={"age_11": "Age"})
filtered_df_1819["Age"] = filtered_df_1819["Age"].astype(str)
filtered_df_1819 = filtered_df_1819.join(
    filtered_df_1819["Age"].str.get_dummies(sep=",").add_prefix("Age_")
)

# 2019-20
filtered_df_1920 = filtered_df_1920.rename(columns={"age_11": "Age"})
filtered_df_1920["Age"] = filtered_df_1920["Age"].astype(str)
filtered_df_1920 = filtered_df_1920.join(
    filtered_df_1920["Age"].str.get_dummies(sep=",").add_prefix("Age_")
)


In [ ]:
# 2020-21
filtered_df_2021 = filtered_df_2021.rename(columns={"age_11": "Age"})
filtered_df_2021["Age"] = filtered_df_2021["Age"].astype(str)
filtered_df_2021 = filtered_df_2021.join(
    filtered_df_2021["Age"].str.get_dummies(sep=",").add_prefix("Age_")
)

# 2021-22
filtered_df_2122 = filtered_df_2122.rename(columns={"age_11": "Age"})
filtered_df_2122["Age"] = filtered_df_2122["Age"].astype(str)
filtered_df_2122 = filtered_df_2122.join(
    filtered_df_2122["Age"].str.get_dummies(sep=",").add_prefix("Age_")
)

# 2022-23
filtered_df_2223 = filtered_df_2223.rename(columns={"age_11": "Age"})
filtered_df_2223["Age"] = filtered_df_2223["Age"].astype(str)
filtered_df_2223 = filtered_df_2223.join(
    filtered_df_2223["Age"].str.get_dummies(sep=",").add_prefix("Age_")
)


# Function 5: Common Functions for preprocessing

In [ ]:
def age(data):
    a=data[data[age_column]==1]  # Or any other number that the data might represent the age category.
    return a                     # In our case, "1" means age category 16-19.

def males(data):
    b=data[data[gender_column]==1] # Or any other number that the data might represent as male

def females(data):
    b=data[data[gender_column]==2] 
    
def sport(data):
    b=data[data[sport_column]==1] # We have already filtered the rugby column which has values, "0" meaning No, "1" meaning yes to playing rugby or not.
    return b

In [ ]:
Sport_1718 = sport(filtered_df_1718)
Sport_1819 = sport(filtered_df_1819)
Sport_1920 = sport(filtered_df_1920)

In [ ]:
Sport_2021=sport(filtered_df_2021)
Sport_2122=sport(filtered_df_2122)
Sport_2223=sport(filtered_df_2223)

# Function 6: convert_to_desired_format()

In [ ]:
import os

def convert_to_desired_format(data, desired_fileformat='csv', desired_file_name=None):
     # Default file name if none is provided
    if desired_file_name is None:
        desired_file_name = f"output.{desired_fileformat}"
    
    # Convert and save in the required format
    if desired_fileformat.lower() == 'csv':
        data.to_csv(desired_file_name, index=False)
    elif desired_fileformat.lower() == 'json':
        data.to_json(desired_file_name, orient="records", indent=4)
    elif desired_fileformat.lower() in ['xls', 'xlsx', 'excel']:
        data.to_excel(desired_file_name, index=False, engine="openpyxl")
    else:
        raise ValueError("Unsupported file format. Choose from: 'csv', 'json', 'excel'.")

    print(f"Data saved as {desired_file_name}")

In [ ]:
convert_to_desired_format(Sport_1718, desired_fileformat='csv', desired_file_name="Sport_17_18_young_adults_new.csv")
convert_to_desired_format(Sport_1819, desired_fileformat='csv', desired_file_name="Sport_18_19_young_adults_new.csv")
convert_to_desired_format(Sport_1920, desired_fileformat='csv', desired_file_name="Sport_19_20_young_adults_new.csv")

In [ ]:
convert_to_desired_format(Sport_2021, desired_fileformat='csv', desired_file_name="Sport_20_21_young_adults_new.csv")
convert_to_desired_format(Sport_2122, desired_fileformat='csv', desired_file_name="Sport_21_22_young_adults_new.csv")
convert_to_desired_format(Sport_2223, desired_fileformat='csv', desired_file_name="Sport_22_23_young_adults_new.csv")

# Extract RFU files using function 1

In [ ]:
file_path_rfu=["RFU_data.xlsx"]

In [ ]:
df_1,df_2,df_3,df_4,df_5=load_files(file_path_rfu, sheet_indices=[7,9,11,13,15])

In [ ]:
df1 = df_1.iloc[:-1, :-7] # This step is done to filter out age 18 plus entries
df2 = df_2.iloc[:-1, :-7]
df3 = df_3.iloc[:-1, :-7]
df4 = df_4.iloc[:-1, :-7]
df5 = df_5.iloc[:-1, :-7]

In [ ]:
'''
Years 2011, 2016,2017....2023
'''
dfs = [df1, df2, df3, df4, df5]
# for df in dfs:
#     df.drop('Unnamed: 32', axis=1, inplace=True)
    
years_rfu= ["2018", "2019", "2019 nov", "2021 Jan", "2023 May"]

# Extract Saved ALS files using function 1

In [ ]:
als_datasets=['Sport_17_18_young_adults_new.csv', "Sport_18_19_young_adults_new.csv", "Sport_19_20_young_adults_new.csv", 
         "Sport_20_21_young_adults_new.csv", "Sport_21_22_young_adults_new.csv", "Sport_22_23_young_adults_new.csv"]

In [ ]:
df_2017_18, df_2018_19, df_2019_20, df_2020_21, df_2021_22, df_2022_23=load_files(als_datasets)

# Function 8: weight_calc() [Only for ALS Data],[Skip for RFU]

In [ ]:
generic_columns=["Respondent_Serial","wt_gross"]
gender_column='gend3'
sport_column = "onceawk_modplus_everywhere_GR_RUGBY_CC018"
age_column = "Age"

In [ ]:
def weight_calc(df, col_1, col_2="wt_gross"):
    df["weighted_total"] = df[col_1] * df[col_2]
    return df

In [ ]:
'''
Sport column variable (replace with actual column name)
'''
#sport_column = "Sport_Column_Name"

# Apply weight_calc to all datasets
df_2017_18 = weight_calc(df_2017_18, col_1=sport_column)
df_2018_19 = weight_calc(df_2018_19, col_1=sport_column)
df_2019_20 = weight_calc(df_2019_20, col_1=sport_column)
df_2020_21 = weight_calc(df_2020_21, col_1=sport_column)
df_2021_22 = weight_calc(df_2021_22, col_1=sport_column)
df_2022_23 = weight_calc(df_2022_23, col_1=sport_column)


# Function 9: split_by_gender()

In [ ]:
def split_by_gender(df,gender_column='gend3', male_code=1, female_code=2):
    df_male = df[df[gender_column] == male_code]
    df_female = df[df[gender_column] == female_code]
    return df_male, df_female

In [ ]:
# Splitting each dataset into male & female
df_2017_18_male, df_2017_18_female = split_by_gender(df_2017_18)
df_2018_19_male, df_2018_19_female = split_by_gender(df_2018_19)
df_2019_20_male, df_2019_20_female = split_by_gender(df_2019_20)
df_2020_21_male, df_2020_21_female = split_by_gender(df_2020_21)
df_2021_22_male, df_2021_22_female = split_by_gender(df_2021_22)
df_2022_23_male, df_2022_23_female = split_by_gender(df_2022_23)

In [ ]:
# Weighted counts for male
df_2017_18_male = weight_calc(df_2017_18_male, col_1=sport_column)
df_2018_19_male = weight_calc(df_2018_19_male, col_1=sport_column)
df_2019_20_male = weight_calc(df_2019_20_male, col_1=sport_column)
df_2020_21_male = weight_calc(df_2020_21_male, col_1=sport_column)
df_2021_22_male = weight_calc(df_2021_22_male, col_1=sport_column)
df_2022_23_male = weight_calc(df_2022_23_male, col_1=sport_column)

# Weighted counts for female
df_2017_18_female = weight_calc(df_2017_18_female, col_1=sport_column)
df_2018_19_female = weight_calc(df_2018_19_female, col_1=sport_column)
df_2019_20_female = weight_calc(df_2019_20_female, col_1=sport_column)
df_2020_21_female = weight_calc(df_2020_21_female, col_1=sport_column)
df_2021_22_female = weight_calc(df_2021_22_female, col_1=sport_column)
df_2022_23_female = weight_calc(df_2022_23_female, col_1=sport_column)

# Labelling for ALS and RFU

In [ ]:
weighted_column="weighted_total"
datasets = [df_2017_18, df_2018_19, df_2019_20, df_2020_21, df_2021_22, df_2022_23]
years_als = ["2017-18", "2018-19", "2019-20", "2020-21", "2021-22", "2022-23"]

# List of datasets
male_datasets = [df_2017_18_male, df_2018_19_male, df_2019_20_male, df_2020_21_male, df_2021_22_male, df_2022_23_male]

# List of datasets
female_datasets = [df_2017_18_female, df_2018_19_female, df_2019_20_female, df_2020_21_female, df_2021_22_female, df_2022_23_female]


In [ ]:
# Loop through and add 'count_col' to each
for i, df in enumerate(dfs):
    count_cols = [col for col in df.columns if col.endswith("M") or col.endswith("F")]
    dfs[i]["count_col"] = df[count_cols].sum(axis=1)
    
for i, df in enumerate(dfs):
    count_colsm = [col for col in df.columns if col.endswith("M")]
    dfs[i]["count_col_m"] = df[count_colsm].sum(axis=1)

for i, df in enumerate(dfs):
    count_colsf = [col for col in df.columns if col.endswith("F")]
    dfs[i]["count_col_f"] = df[count_colsf].sum(axis=1)

# Function 10: ALS_overall_plot

In [ ]:
def als_overall_plot(datasets, years):
    overall_sums = [df["weighted_total"].sum() for df in datasets]

    # Plot
    print(overall_sums)
    plt.figure(figsize=(10, 5))
    plt.plot(years, overall_sums, marker='o', linestyle='-', label="Overall Participation Count (Weighted Sum)", color='blue')
    plt.xlabel("Year")
    plt.ylabel("Weighted Total")
    plt.title("Year-wise Overall (Males plus Females Combined) Trend (Weighted Total)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return pd.DataFrame({
        "Year": years,
        "Total_Count": overall_sums,
    })


In [ ]:
als_overall_plot(datasets, years_als)

# Function 11: plot_weighted_gender_trend() for ALS

In [ ]:
import matplotlib.pyplot as plt

# Function to create year-wise line plot for Male & Female trends
def als_genderwise_plot(male_datasets, female_datasets, years):
    male_sums = [df["weighted_total"].sum() for df in male_datasets]
    female_sums = [df["weighted_total"].sum() for df in female_datasets]

    # Plot
    plt.figure(figsize=(10, 5))
    plt.plot(years, male_sums, marker='o', linestyle='-', label="Male (Weighted Sum)", color='blue')
    plt.plot(years, female_sums, marker='o', linestyle='-', label="Female (Weighted Sum)", color='red')

    plt.xlabel("Year")
    plt.ylabel("Weighted Total")
    plt.title("Year-wise Male & Female Trend (Weighted Total)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return pd.DataFrame({
        "Year": years,
        "Total_Male": male_sums,
        "Total_Female": female_sums
    })


In [ ]:
als_genderwise_plot(male_datasets, female_datasets, years_als)

# Function 12: Total RFU y-o-y participation

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def rfu_overall_plot(dfs, years, title="Total Participation (Male + Female) Over Years"):
    """
    Plots total (male + female) participation per year.

    Parameters:
    dfs (list of pd.DataFrame): List of yearly DataFrames.
    years (list): Corresponding year labels.
    title (str): Plot title.
    """

    total_combined = []

    for df in dfs:
        male_cols = [col for col in df.columns if col.endswith('M')]
        female_cols = [col for col in df.columns if col.endswith('F')]

        male_total = df[male_cols].sum().sum()
        female_total = df[female_cols].sum().sum()
        total_combined.append(male_total + female_total)

    # Plotting
    plt.figure(figsize=(8, 5))
    plt.plot(years, total_combined, marker='^', linestyle='--', color='purple', label='Total (M + F)')
    plt.title(title)
    plt.xlabel("Year")
    plt.ylabel("Total Participants")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    return pd.DataFrame({
        "Year": years,
        "Total_Count": total_combined,
    })


In [ ]:
rfu_overall_plot(dfs, years_rfu)

# Function 13: Genderwise yoy RFU participation

In [ ]:
#copy
import matplotlib.pyplot as plt
import pandas as pd

def rfu_gender_plot(dfs, years, title="Year-wise Participation by Gender for RFU(U17-U17)"):

    total_males = []
    total_females = []

    for df in dfs:
        # Filter only columns ending with 'M' or 'F' and NOT starting with 'count_col'
        male_cols = [col for col in df.columns if col.endswith('M')]
        female_cols = [col for col in df.columns if col.endswith('F')]

        male_total = df[male_cols].sum().sum()                        
        female_total = df[female_cols].sum().sum()                    
       
        total_males.append(male_total)
        total_females.append(female_total)

    # Plotting
    plt.figure(figsize=(8, 5))
    plt.plot(years, total_males, marker='o', label='Male')
    plt.plot(years, total_females, marker='s', label='Female')
    plt.title(title)
    plt.xlabel("Year")
    plt.ylabel("Total Participants")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Summary table
    return pd.DataFrame({
        "Year": years,
        "Total_Male": total_males,
        "Total_Female": total_females
    })


In [ ]:
rfu_gender_plot(dfs, years_rfu)

# Function 14: group_by_columns()

In [ ]:
import pandas as pd

def group_by_columns(df, years, groupby_cols, count_col):

    grouped_data = {}

    for df, year in zip(df, years):
        grouped_df = df.groupby(groupby_cols)[count_col].sum()
        grouped_data[year] = grouped_df.copy()  # Ensure a copy is stored
    df_1=pd.DataFrame(grouped_data)
    return df_1


In [ ]:
'''
Implementing Function 10
'''
weighted_column="weighted_total"
regionwise_males=group_by_columns(male_datasets, years_als, groupby_cols=['Region','CSP'], count_col=weighted_column)
regionwise_females=group_by_columns(female_datasets, years_als, groupby_cols=['Region','CSP'], count_col=weighted_column)
regionwise_overall=group_by_columns(datasets, years_als, groupby_cols=['Region','CSP'], count_col=weighted_column)

In [ ]:
males_regionwise_rfu=group_by_columns(dfs, years_rfu, groupby_cols=['Constituent Body'], count_col="count_col_m")
females_regionwise_rfu=group_by_columns(dfs, years_rfu, groupby_cols=['Constituent Body'], count_col="count_col_f")
overall_regionwise_rfu=group_by_columns(dfs, years_rfu, groupby_cols=['Constituent Body'], count_col="count_col")

## Removing counties containing zero values

In [ ]:
overall_regionwise_rfu_nonzero_filtered = overall_regionwise_rfu[(overall_regionwise_rfu != 0).all(axis=1)]
overall_regionwise_rfu_zero_filtered= overall_regionwise_rfu[(overall_regionwise_rfu == 0).all(axis=1)]

males_regionwise_rfu_nonzero_filtered = males_regionwise_rfu[(males_regionwise_rfu != 0).any(axis=1)]
males_regionwise_rfu_zero_filtered=males_regionwise_rfu[(males_regionwise_rfu == 0).any(axis=1)]

females_regionwise_rfu_nonzero_filtered = females_regionwise_rfu[(females_regionwise_rfu != 0).any(axis=1)]
females_regionwise_rfu_zero_filtered= females_regionwise_rfu[(females_regionwise_rfu == 0).any(axis=1)]

In [ ]:
for year, df in zip(years_als, datasets):
    total = df['weighted_total'].sum()
    print(f"Dataset {year}: {total:,.2f}")

for year, df in zip(years_als, male_datasets):
    total = df['weighted_total'].sum()
    print(f"Dataset {year} (Male): {total:,.2f}")

for year, df in zip(years_als, female_datasets):
    total = df['weighted_total'].sum()
    print(f"Dataset {year} (Female): {total:,.2f}")


# Function 15: Plots for various modes for ALS and RFU

In [ ]:
#DONE CODE
import matplotlib.pyplot as plt

def plot_category_trends(df, top_n=5, mode="largest", first_col=None, last_col=None,
                         xlabel="Year", ylabel="Participation Count", title="Participation Trends"):

    """
    Plots year-wise trends for the top OR bottom N categories based on selected analysis modes.
    Modes supported: largest, most_improved_custom, least_improved_custom,
                     cagr, lowest_cagr, recovery, weakest_recovery

    Parameters:
    df (DataFrame): Rows = categories, Columns = time points.
    top_n (int): Top N categories to plot.
    mode (str): Analysis mode.
    first_col (str): Starting year/column name.
    last_col (str): Ending year/column name.
    xlabel (str), ylabel (str), title (str): Plot labels.

    Returns:
    DataFrame: Selected categories and their summary values.
    """

    if mode == "largest":
        selected_categories = df.sum(axis=1).nlargest(top_n)
        title = f"Top {top_n} {title}"

    elif mode == "smallest":
        selected_categories = df.sum(axis=1).nsmallest(top_n)
        title = f"Top {top_n} {title}"

    elif mode == "most_improved_custom":
        improvement = df[last_col] - df[first_col]
        selected_categories = improvement.nlargest(top_n)
        title = f"Most Improved {top_n} {title}"

    elif mode == "least_improved_custom":
        improvement = df[last_col] - df[first_col]
        selected_categories = improvement.nsmallest(top_n)
        title = f"Least Improved {top_n} {title}"

    elif mode == "percent_growth":
        percent_growth = ((df[last_col] - df[first_col]) / df[first_col]) * 100
        selected_categories = percent_growth.nlargest(top_n)
        title = f"Top {top_n} Percent Growth {title}"
        
    elif mode == "percent_fall":
        percent_growth = ((df[last_col] - df[first_col]) / df[first_col]) * 100
        selected_categories = percent_growth.nsmallest(top_n)
        title = f"Top {top_n} Percent Fall {title}"

    elif mode == "recovery":
        middle_cols = df.loc[:, first_col:last_col].iloc[:, 1:-1].columns
        min_dip = df[middle_cols].min(axis=1)
        recovery_strength = df[last_col] - min_dip
        selected_categories = recovery_strength.nlargest(top_n)
        title = f"Top {top_n} Recovery after Dip {title}"

    elif mode == "weakest_recovery":
        middle_cols = df.loc[:, first_col:last_col].iloc[:, 1:-1].columns
        min_dip = df[middle_cols].min(axis=1)
        recovery_strength = df[last_col] - min_dip
        selected_categories = recovery_strength.nsmallest(top_n)
        title = f"Weakest {top_n} Recovery after Dip {title}"

    # ✅ Plot
    for category in selected_categories.index:
        plt.plot(df.columns, df.loc[category], marker='o', linestyle='-', label=category)

        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.title(title)
        plt.xticks(rotation=45)
        plt.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize="small")
        plt.grid(True)

    return selected_categories.to_frame(name='Summary Value')


# Overall ALS Plots

In [ ]:


'''
modes = [
    "largest", "smallest", "most_improved_custom", "least_improved_custom", "cagr",
    "lowest_cagr", "recovery", "weakest_recovery"
]
'''

In [ ]:
first_col="2017-18"
last_col='2022-23'

In [ ]:
plot_category_trends(regionwise_overall, first_col="2017-18",last_col='2022-23',mode='largest')

In [ ]:
plot_category_trends(regionwise_overall, first_col="2017-18",last_col='2022-23',mode='smallest')

In [ ]:
plot_category_trends(regionwise_overall, first_col="2017-18",last_col='2022-23',mode='most_improved_custom')

In [ ]:
plot_category_trends(regionwise_overall, first_col="2017-18",last_col='2022-23', mode='least_improved_custom')

In [ ]:
plot_category_trends(regionwise_overall,  first_col="2017-18",last_col='2022-23',  mode='percent_growth')

In [ ]:
plot_category_trends(regionwise_overall,  first_col="2017-18",last_col='2022-23',  mode='percent_fall')

# Overall RFU Plots

In [ ]:
plot_category_trends(overall_regionwise_rfu_nonzero_filtered, first_col='2018', last_col='2023 May', mode='largest')

In [ ]:
plot_category_trends(overall_regionwise_rfu_nonzero_filtered, top_n=8, first_col='2018', last_col='2023 May', mode='smallest')

In [ ]:
plot_category_trends(overall_regionwise_rfu_nonzero_filtered, first_col='2018', last_col='2023 May', mode='most_improved_custom')

In [ ]:
plot_category_trends(overall_regionwise_rfu_nonzero_filtered, top_n=8, first_col='2018', last_col='2023 May', mode='least_improved_custom')

In [ ]:
plot_category_trends(overall_regionwise_rfu_nonzero_filtered, first_col='2018', last_col='2023 May', mode='percent_growth')

In [ ]:
plot_category_trends(overall_regionwise_rfu_nonzero_filtered, top_n=8,  first_col='2018', last_col='2023 May', mode='percent_fall')

# ALS Plots (Males)

In [ ]:
first_col="2017-18"
last_col='2022-23'

'''
modes = [
    "largest", "smallest", "most_improved_custom", "least_improved_custom", "cagr",
    "lowest_cagr", "recovery", "weakest_/'recovery"
]
'''

In [ ]:
plot_category_trends(regionwise_males,  first_col="2017-18",last_col='2022-23',  mode='largest')

In [ ]:
plot_category_trends(regionwise_males, first_col="2017-18",last_col='2022-23', mode='smallest')

In [ ]:
plot_category_trends(regionwise_males,first_col="2017-18",last_col='2022-23', mode='most_improved_custom')

In [ ]:
plot_category_trends(regionwise_males,first_col="2017-18",last_col='2022-23', mode='least_improved_custom')

In [ ]:
plot_category_trends(regionwise_males,first_col="2017-18",last_col='2022-23', mode='percent_growth')

In [ ]:
plot_category_trends(regionwise_males,first_col="2017-18",last_col='2022-23', mode='percent_fall')

# RFU Plots (Males)

In [ ]:
'''
modes = [
    "largest", "smallest", "most_improved_custom", "least_improved_custom", "percent_growth",
    "percent_fall"
]
'''

In [ ]:
plot_category_trends(males_regionwise_rfu_nonzero_filtered,  first_col='2018', last_col='2023 May', mode='largest')

In [ ]:
plot_category_trends(males_regionwise_rfu_nonzero_filtered, top_n=8,  first_col='2018', last_col='2023 May', mode='smallest')

In [ ]:
plot_category_trends(males_regionwise_rfu_nonzero_filtered, top_n=8,  first_col='2018', last_col='2023 May', mode='most_improved_custom')

In [ ]:
plot_category_trends(males_regionwise_rfu_nonzero_filtered, top_n=8, first_col='2018', last_col='2023 May', mode='least_improved_custom')

In [ ]:
plot_category_trends(males_regionwise_rfu_nonzero_filtered, first_col='2018', last_col='2023 May', mode='percent_growth')

In [ ]:
plot_category_trends(males_regionwise_rfu_nonzero_filtered, top_n=8, first_col='2018', last_col='2023 May', mode='percent_fall')

# ALS Plots (Females)

In [ ]:
plot_category_trends(regionwise_females,  first_col="2017-18",last_col='2022-23',  mode='largest')

In [ ]:
plot_category_trends(regionwise_females,  first_col="2017-18",last_col='2022-23',  mode='smallest')

In [ ]:
plot_category_trends(regionwise_females,  first_col="2017-18",last_col='2022-23',  mode='most_improved_custom')

In [ ]:
plot_category_trends(regionwise_females,  first_col="2017-18",last_col='2022-23',  mode='least_improved_custom')

In [ ]:
plot_category_trends(regionwise_females,  first_col="2017-18",last_col='2022-23',  mode='percent_growth')

In [ ]:
plot_category_trends(regionwise_females,  first_col="2017-18",last_col='2022-23',  mode='percent_fall')

# RFU Plots (Females)

In [ ]:
plot_category_trends(females_regionwise_rfu_nonzero_filtered, first_col='2018', last_col='2023 May', mode='largest')

In [ ]:
plot_category_trends(females_regionwise_rfu_nonzero_filtered, top_n=8, first_col='2018', last_col='2023 May', mode='smallest')

In [ ]:
plot_category_trends(females_regionwise_rfu_nonzero_filtered, top_n=8,first_col='2018', last_col='2023 May', mode='most_improved_custom')

In [ ]:
plot_category_trends(females_regionwise_rfu_nonzero_filtered, top_n=8, first_col='2018', last_col='2023 May', mode='least_improved_custom')

In [ ]:
plot_category_trends(females_regionwise_rfu_nonzero_filtered,top_n=8, first_col='2018', last_col='2023 May', mode='percent_growth')

In [ ]:
plot_category_trends(females_regionwise_rfu_nonzero_filtered, top_n=8, first_col='2018', last_col='2023 May', mode='percent_fall')

# Age-ALS-Overall

In [ ]:
datasets = [df_2017_18, df_2018_19, df_2019_20, df_2020_21, df_2021_22, df_2022_23]
years = ["2017-18", "2018-19", "2019-20", "2020-21", "2021-22", "2022-23"]

In [ ]:
agew_over=group_by_columns(datasets, years, groupby_cols=['Age'], count_col=sport_column)

In [ ]:
plot_category_trends(agew_over)

In [ ]:
agew_over=group_by_columns(male_datasets, years, groupby_cols=['Age'], count_col=sport_column)

In [ ]:
plot_category_trends(agew_over)

In [ ]:
agew_over=group_by_columns(female_datasets, years, groupby_cols=['Age'], count_col=sport_column)

In [ ]:
plot_category_trends(agew_over)

# Age-RFU-Overall

In [ ]:
age_columns_rfu = ['U7M', 'U7F', 'U8M', 'U8F', 'U9M', 'U9F', 'U10M', 'U10F',
                   'U11M', 'U11F', 'U12M', 'U12F', 'U13M', 'U13F', 'U14M', 'U14F',
                   'U15M', 'U15F', 'U16M', 'U16F', 'U17M','U17F']

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_top5_total_age_groups(dfs, years, age_columns):
    total_trend = {}

    for df, year in zip(dfs, years):
        yearly_totals = {}
        for col in age_columns:
            base_age = col[:-1]  # strip 'M' or 'F'
            male_col = base_age + 'M'
            female_col = base_age + 'F'

            if male_col in df.columns and female_col in df.columns:
                total = df[[male_col, female_col]].sum().sum()
                yearly_totals[base_age] = total

        total_trend[year] = yearly_totals

    total_df = pd.DataFrame(total_trend).T

    # Get top 5 by average participation
    top5_ages = total_df.mean().sort_values(ascending=False).head(5).index.tolist()

    # Plot
    plt.figure(figsize=(10, 6))
    for age in top5_ages:
        plt.plot(total_df.index, total_df[age], marker='o', label=age)

    plt.title("Top 5 Age Groups by Total Participation (M + F)")
    plt.xlabel("Year")
    plt.ylabel("Participants")
    plt.legend(title="Age Group", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return total_df[top5_ages]


In [ ]:
plot_top5_total_age_groups(dfs, years, age_columns=age_columns_rfu)

In [ ]:
def analyze_top5_overall_age_groups(dfs, years, age_columns):
    overall_trend = {}

    for df, year in zip(dfs, years):
        overall_totals = df[age_columns].sum(axis=0)
        overall_trend[year] = overall_totals

    overall_df = pd.DataFrame(overall_trend).T
    top_5_overall = overall_df.mean().sort_values(ascending=False).head(5).index.tolist()

    plt.figure(figsize=(10, 6))
    for col in top_5_overall:
        plt.plot(overall_df.index, overall_df[col], marker='o', label=col)
    plt.title("Top 5 Overall Age Groups by Participation (Year-on-Year)")
    plt.xlabel("Year")
    plt.ylabel("Participants")
    plt.legend(title="Age Group")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return overall_df

def analyze_top5_male_age_groups(dfs, years, age_columns):
    male_cols = [col for col in age_columns if col.endswith('M')]
    male_trend = {}

    for df, year in zip(dfs, years):
        male_totals = df[male_cols].sum(axis=0)
        male_trend[year] = male_totals

    male_df = pd.DataFrame(male_trend).T
    top_5_males = male_df.mean().sort_values(ascending=False).head(5).index.tolist()

    plt.figure(figsize=(10, 6))
    for col in top_5_males:
        plt.plot(male_df.index, male_df[col], marker='o', label=col)
    plt.title("Top 5 Male Age Groups by Participation (Year-on-Year)")
    plt.xlabel("Year")
    plt.ylabel("Participants")
    plt.legend(title="Age Group")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return male_df
    
def analyze_top5_female_age_groups(dfs, years, age_columns):
    female_cols = [col for col in age_columns if col.endswith('F')]
    female_trend = {}

    for df, year in zip(dfs, years):
        female_totals = df[female_cols].sum(axis=0)
        female_trend[year] = female_totals

    female_df = pd.DataFrame(female_trend).T
    top_5_females = female_df.mean().sort_values(ascending=False).head(5).index.tolist()

    plt.figure(figsize=(10, 6))
    for col in top_5_females:
        plt.plot(female_df.index, female_df[col], marker='o', label=col)
    plt.title("Top 5 Female Age Groups by Participation (Year-on-Year)")
    plt.xlabel("Year")
    plt.ylabel("Participants")
    plt.legend(title="Age Group")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return female_df


In [ ]:
analyze_top5_overall_age_groups(dfs, years, age_columns_rfu)

In [ ]:
analyze_top5_male_age_groups(dfs, years, age_columns_rfu)

In [ ]:
analyze_top5_female_age_groups(dfs, years, age_columns_rfu)